In [1]:
import tushare as ts
import pandas as pd
import pymysql
import baostock as bs
import time

In [2]:
hs300_ts = ts.get_hs300s()
hs300_ts.head()

,date,code,name,weight
0,2019-09-30,600000,浦发银行,1.12
1,2019-09-30,600004,白云机场,0.19
2,2019-09-30,600009,上海机场,0.62
3,2019-09-30,600010,包钢股份,0.22
4,2019-09-30,600011,华能国际,0.21


In [3]:
lg = bs.login()
rs = bs.query_hs300_stocks()
hs300_stocks = []
while (rs.error_code == '0') & rs.next():
    hs300_stocks.append(rs.get_row_data())
hs300_bs = pd.DataFrame(hs300_stocks, columns=rs.fields)
bs.logout()
hs300_bs.head()

login success!
logout success!


,updateDate,code,code_name
0,2019-09-23,sh.600000,浦发银行
1,2019-09-23,sh.600004,白云机场
2,2019-09-23,sh.600009,上海机场
3,2019-09-23,sh.600010,包钢股份
4,2019-09-23,sh.600011,华能国际


In [4]:
hs300_bs.rename(columns={'code':'bs_code'}, inplace=True)
hs300_bs['code'] = ''
for i in range(0,len(hs300_bs['bs_code'])):
    hs300_bs['code'][i] = str(hs300_bs['bs_code'][i])[3:]

In [5]:
hs300_bs.head()

,updateDate,bs_code,code_name,code
0,2019-09-23,sh.600000,浦发银行,600000
1,2019-09-23,sh.600004,白云机场,600004
2,2019-09-23,sh.600009,上海机场,600009
3,2019-09-23,sh.600010,包钢股份,600010
4,2019-09-23,sh.600011,华能国际,600011


In [6]:
hs300 = pd.merge(hs300_bs,hs300_ts,on=['code'], how='right')#left
hs300.tail()

,updateDate,bs_code,code_name,code,date,name,weight
295,2019-09-23,sz.300296,利亚德,300296,2019-09-30,利亚德,0.10
296,2019-09-23,sz.300408,三环集团,300408,2019-09-30,三环集团,0.17
297,2019-09-23,sz.300413,芒果超媒,300413,2019-09-30,芒果超媒,0.08
298,2019-09-23,sz.300433,蓝思科技,300433,2019-09-30,蓝思科技,0.07
299,2019-09-23,sz.300498,温氏股份,300498,2019-09-30,温氏股份,1.12


In [7]:
def DB_Select(code):
    
    #### 连接database ####
    # conn = pymysql.connect(host=“你的数据库地址”, user=“用户名”,password=“密码”,database=“数据库名”,charset=“utf8”)
    conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='123456' ,charset='utf8')
    # 使用cursor()方法获取操作游标，cursor()得到一个可以执行SQL语句的光标对象
    cursor = conn.cursor()
    
    #### 选择数据库：stock ####
    Select_DataBase = "use stock;"
    cursor.execute(Select_DataBase)
    table_name = code
    
    sql_select = """select * from %s"""% table_name
    
    df = pd.read_sql(sql_select, con=conn)
    
    # 关闭光标对象
    cursor.close()
    # 关闭数据库连接
    conn.close()
    
    return df

In [8]:
stock_basic = DB_Select('stock_basic')
stock_basic.rename(columns={'code':'bs_code','code_name':'code_name_sb'}, inplace=True)
stock_basic.head()

,id,bs_code,code_name_sb,ipoDate,outDate,stocktype,tradeStatus
0,1,sh.000001,上证综合指数,1991-07-15,,2,1
1,2,sh.000002,上证A股指数,1992-02-21,,2,1
2,3,sh.000003,上证B股指数,1992-08-17,,2,1
3,4,sh.000004,上证工业类指数,1993-05-03,,2,1
4,5,sh.000005,上证商业类指数,1993-05-03,,2,1


In [9]:
hs300 = pd.merge(hs300,stock_basic,on=['bs_code'], how='left')#right
hs300.tail()

,updateDate,bs_code,code_name,code,date,name,weight,id,code_name_sb,ipoDate,outDate,stocktype,tradeStatus
295,2019-09-23,sz.300296,利亚德,300296,2019-09-30,利亚德,0.10,3542,利亚德,2012-03-15,,1,1
296,2019-09-23,sz.300408,三环集团,300408,2019-09-30,三环集团,0.17,3653,三环集团,2014-12-03,,1,1
297,2019-09-23,sz.300413,芒果超媒,300413,2019-09-30,芒果超媒,0.08,3658,芒果超媒,2015-01-21,,1,1
298,2019-09-23,sz.300433,蓝思科技,300433,2019-09-30,蓝思科技,0.07,3678,蓝思科技,2015-03-18,,1,1
299,2019-09-23,sz.300498,温氏股份,300498,2019-09-30,温氏股份,1.12,3743,温氏股份,2015-11-02,,1,1


In [10]:
hs300['startdate'] = ''

In [11]:
for j in range(0,len(hs300['ipoDate'])):
    hs300.loc[j,'startdate'] = str(hs300['ipoDate'][j])[0:4]+'-01-01'

In [12]:
hs300.head()

,updateDate,bs_code,code_name,code,date,name,weight,id,code_name_sb,ipoDate,outDate,stocktype,tradeStatus,startdate
0,2019-09-23,sh.600000,浦发银行,600000,2019-09-30,浦发银行,1.12,230,浦发银行,1999-11-10,,1,1,1999-01-01
1,2019-09-23,sh.600004,白云机场,600004,2019-09-30,白云机场,0.19,234,白云机场,2003-04-28,,1,1,2003-01-01
2,2019-09-23,sh.600009,上海机场,600009,2019-09-30,上海机场,0.62,239,上海机场,1998-02-18,,1,1,1998-01-01
3,2019-09-23,sh.600010,包钢股份,600010,2019-09-30,包钢股份,0.22,240,包钢股份,2001-03-09,,1,1,2001-01-01
4,2019-09-23,sh.600011,华能国际,600011,2019-09-30,华能国际,0.21,241,华能国际,2001-12-06,,1,1,2001-01-01


In [13]:
hs300['ts_TradeCount'] = ''
cons = ts.get_apis()

In [14]:
for k in range(0,len(hs300['startdate'])):
    print(k,hs300['code'][k],hs300['name'][k])
    code = hs300['code'][k]
    hs300_single = ts.bar(code, conn=cons)
    hs300.loc[k,'ts_TradeCount'] = len(hs300_single['code'])
    print(len(hs300_single['code']),'\n')

0 600000 浦发银行
4692 

1 600004 白云机场
3933 

2 600009 上海机场
5190 

3 600010 包钢股份
4378 

4 600011 华能国际
4243 

5 600015 华夏银行
3792 

6 600016 民生银行
4481 

7 600018 上港集团
4511 

8 600019 宝钢股份
4393 

9 600023 浙能电力
1412 

10 600025 华能水电
437 

11 600027 华电国际
3504 

12 600028 中国石化
4342 

13 600029 南方航空
3823 

14 600030 中信证券
3965 

15 600031 三一重工
3891 

16 600036 招商银行
4164 

17 600038 中直股份
4390 

18 600048 保利地产
3181 

19 600050 中国联通
3977 

20 600061 国投资本
5217 

21 600066 宇通客车
5331 

22 600068 葛洲坝
5297 

23 600085 同仁堂
5349 

24 600089 特变电工
5206 

25 600100 同方股份
5154 

26 600104 上汽集团
5172 

27 600109 国金证券
5192 

28 600111 北方稀土
5271 

29 600115 东方航空
5099 

30 600118 中国卫星
5269 

31 600153 建发股份
4859 

32 600170 上海建工
5035 

33 600176 中国巨石
4835 

34 600177 雅戈尔
4922 

35 600188 兖州煤业
5067 

36 600196 复星医药
5033 

37 600208 新湖中宝
4716 

38 600219 南山铝业
4611 

39 600221 海航控股
4536 

40 600233 圆通速递
4540 

41 600271 航天信息
3746 

42 600276 恒瑞医药
4542 

43 600297 广汇汽车
4222 

44 600299 安迪苏
4432 

45 600309 万华化学
4337 

46 

In [15]:
ts.close_apis(cons)

In [16]:
hs300 = hs300[['updateDate','bs_code','date','code','code_name','weight','ipoDate','tradeStatus','startdate','ts_TradeCount']]
hs300

,updateDate,bs_code,date,code,code_name,weight,ipoDate,tradeStatus,startdate,ts_TradeCount
0,2019-09-23,sh.600000,2019-09-30,600000,浦发银行,1.12,1999-11-10,1,1999-01-01,4692
1,2019-09-23,sh.600004,2019-09-30,600004,白云机场,0.19,2003-04-28,1,2003-01-01,3933
2,2019-09-23,sh.600009,2019-09-30,600009,上海机场,0.62,1998-02-18,1,1998-01-01,5190
3,2019-09-23,sh.600010,2019-09-30,600010,包钢股份,0.22,2001-03-09,1,2001-01-01,4378
4,2019-09-23,sh.600011,2019-09-30,600011,华能国际,0.21,2001-12-06,1,2001-01-01,4243
5,2019-09-23,sh.600015,2019-09-30,600015,华夏银行,0.37,2003-09-12,1,2003-01-01,3792
6,2019-09-23,sh.600016,2019-09-30,600016,民生银行,1.21,2000-12-19,1,2000-01-01,4481
7,2019-09-23,sh.600018,2019-09-30,600018,上港集团,0.15,2006-10-26,1,2006-01-01,4511
8,2019-09-23,sh.600019,2019-09-30,600019,宝钢股份,0.43,2000-12-12,1,2000-01-01,4393
9,2019-09-23,sh.600023,2019-09-30,600023,浙能电力,0.13,2013-12-19,1,2013-01-01,1412


In [17]:
# def database_stock(stock):
#     #### 连接database ####
#     # conn = pymysql.connect(host=“你的数据库地址”, user=“用户名”,password=“密码”,database=“数据库名”,charset=“utf8”)
#     conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='123456' ,charset='utf8')
#     # 使用cursor()方法获取操作游标，cursor()得到一个可以执行SQL语句的光标对象
#     cursor = conn.cursor()
#     #### 选择数据库：stock ####
#     Select_DataBase = "use stock;"
#     cursor.execute(Select_DataBase)
#     table_name = 'hs300'
    
#     print('正在创建数据表：'+table_name)
#     # 定义要执行的SQL语句，预处理语句创建表
#     try:
#         sql_create = """create table %s"""% table_name +""" (
#                         id int auto_increment,
#                         bs_date VARCHAR(20),
#                         bs_code VARCHAR(20),
#                         ts_date VARCHAR(20),
#                         ts_code VARCHAR(20),
#                         stock_name VARCHAR(20),
#                         weight VARCHAR(20),
#                         ipoDate VARCHAR(20),
#                         tradeStatus VARCHAR(10),
#                         startdate VARCHAR(20),
#                         ts_TradeCount VARCHAR(20),
#                         PRIMARY KEY (id),
#                         UNIQUE KEY (bs_date,bs_code,ts_date,ts_code)
#                         )"""
#         # 执行SQL语句
#         cursor.execute(sql_create)
#     except:
#         print(table_name+'数据表已存在！')
        
#     length = len(stock)
#     sql_insert = ''
#     print('数据量：', length)
#     for i in range(0, length):
#         record = tuple(stock.loc[i])
#         if sql_insert == '':
#             #插入数据语句
#             sql_insert = """INSERT INTO  %s"""% table_name +""" (bs_date,bs_code,ts_date,ts_code,stock_name,
#                             weight,ipoDate,tradeStatus,startdate,ts_TradeCount) 
#                             VALUES ('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s')""" % record
#         else:
#             sql_insert = sql_insert + """,('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s')""" % record
        
#     try:
#         # 执行sql语句
#         cursor.execute(sql_insert)
#         # 提交到数据库执行
#         conn.commit()
#     except:
#         # Rollback in case there is any error
#         conn.rollback()
        
#     # 关闭光标对象
#     cursor.close()
#     # 关闭数据库连接
#     conn.close()

In [18]:
# database_stock(hs300)

In [19]:
def database_stock_REPLACE(stock):
    #### 连接database ####
    # conn = pymysql.connect(host=“你的数据库地址”, user=“用户名”,password=“密码”,database=“数据库名”,charset=“utf8”)
    conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='123456' ,charset='utf8')
    # 使用cursor()方法获取操作游标，cursor()得到一个可以执行SQL语句的光标对象
    cursor = conn.cursor()
    #### 选择数据库：stock ####
    Select_DataBase = "use stock;"
    cursor.execute(Select_DataBase)
    table_name = 'hs300'
        
    length = len(stock)
    sql_insert = ''
    print('数据量：', length)
    for i in range(0, length):
        record = tuple(stock.loc[i])
        if sql_insert == '':
            #插入数据语句
            sql_insert = """REPLACE INTO  %s"""% table_name +""" (bs_date,bs_code,ts_date,ts_code,stock_name,
                            weight,ipoDate,tradeStatus,startdate,ts_TradeCount) 
                            VALUES ('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s')""" % record
        else:
            sql_insert = sql_insert + """,('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s')""" % record
        
    try:
        # 执行sql语句
        cursor.execute(sql_insert)
        # 提交到数据库执行
        conn.commit()
    except:
        # Rollback in case there is any error
        conn.rollback()
        
    # 关闭光标对象
    cursor.close()
    # 关闭数据库连接
    conn.close()

In [20]:
database_stock_REPLACE(hs300)

数据量： 300
